In [5]:
import numpy as np
import pandas as pd
import os
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
pd.set_option('display.max_rows', 101)

In [7]:
# Load joined dataframe
df = pd.read_csv('csv/metadata_text_merged_cleaned.csv')

# Frequency analysis

In [8]:
# Get list of words that appear most frequently
freq = pd.Series(' '.join(df['text_cleaned']).split()).value_counts()
dealiased_freq = pd.Series(' '.join(df['text_cleaned_dealiased']).split()).value_counts()

In [16]:
print 'Before:', len(freq), 'After:', len(dealiased_freq)

Before: 17478 After: 16223


In [12]:
freq[:100]

send            2729
sent            2598
shall           2590
mr              2400
one             2390
would           1932
received        1875
two             1839
goods           1747
slaves          1709
per             1665
desire          1650
canoe           1624
captain         1620
may             1577
soe             1343
last            1244
come            1223
wee             1215
great           1215
worship         1206
company         1137
order           1119
much            1113
men             1111
doe             1107
good            1071
att             1046
give            1019
came            1008
people          1006
time            1000
corne            941
pleased          922
accompt          912
royall           912
itt              909
hope             899
dutch            898
noe              885
downe            882
hand             877
instant          871
also             865
hee              857
hath             853
mee              838
since        

In [13]:
dealiased_freq[:100]

send         2731
sent         2606
shall        2591
mr           2400
one          2390
would        1932
recieved     1876
two          1839
canoe        1758
goods        1747
slaves       1709
per          1665
desire       1650
captain      1626
may          1577
so           1343
worship      1292
corn         1281
last         1266
great        1238
come         1223
wee          1215
also         1187
company      1154
order        1119
much         1113
men          1111
do           1107
time         1073
good         1071
at           1046
give         1019
came         1008
peopl        1006
coast         975
cape          955
royal         954
worships      951
dutch         928
pleased       922
accompt       919
it            909
hope          899
noe           885
down          882
hand          880
instant       877
castl         857
hee           857
hath          853
mee           838
sinc          800
want          794
us            793
place         782
upon      